# Instalar librerias

In [ ]:
!pip install opencv2
!pip install numpy
!pip install tensorflow
!pip install mediapipe
!pip install serial

## Importación de Bibliotecas

En esta sección, se importan las bibliotecas necesarias para la funcionalidad del programa:

- `cv2`: Abreviatura de OpenCV (Biblioteca de Visión por Computadora de Código Abierto), una biblioteca ampliamente utilizada para el procesamiento de imágenes y videos en visión por computadora.
- `numpy como np`: Biblioteca para operaciones numéricas y matriciales eficientes.
- `math, time`: Bibliotecas estándar de Python para operaciones matemáticas y manipulación del tiempo, respectivamente.
- `tensorflow como tf`: Biblioteca de aprendizaje profundo utilizada aquí para cargar el modelo de reconocimiento de gestos.
- `load_model` de `tensorflow.keras.models`: Función para cargar modelos preentrenados.
- `mediapipe como mp`: Biblioteca de Google para el seguimiento y análisis de objetos del mundo real, incluido el seguimiento de manos.
- `serial`: Biblioteca para la comunicación serial con dispositivos, como la conexión con Arduino.

In [ ]:
import cv2
import numpy as np
import math, time
import tensorflow as tf
from tensorflow.keras.models import load_model
import mediapipe as mp
import serial
# Set the logging level to suppress verbose output
tf.get_logger().setLevel('INFO')

## Carga del Modelo CNN e Inicializaciónes

Este fragmento de código parece configurar el entorno y los recursos necesarios para el reconocimiento de gestos de mano utilizando un modelo de CNN preentrenado, Mediapipe para el seguimiento de manos y una conexión serial para la interacción con dispositivos externos.

In [ ]:
# Modelo CCN
model = load_model(f'C:\\Users\\pjrio\\projects\\hand-gesture-recognition-code\\mp_mano_gestos')

# Clases
with open('C:\\Users\\pjrio\\projects\\hand-gesture-recognition-code\\gestos.names', 'r') as f:
    classNames = f.read().split('\n')

# Initializar Mediapipe
mp_dibujo = mp.solutions.drawing_utils
mp_manos = mp.solutions.hands

# Initializar la camara 
cap = cv2.VideoCapture(0)


#Variables
enviar_comando = False
ret = 'none'

# Iniciar connecion serial
arduino = serial.Serial('COM10',9600)

## Envío de un Comando a través de Serial

En esta sección, se envía un comando a través de una conexión serial a un dispositivo externo (probablemente Arduino):

```python
comando = 'iniciar'
arduino.write(bytes(comando, 'utf-8'))
time.sleep(0.05)
ret = arduino.readline()
print(ret)
```

- `comando`: Una cadena que contiene el comando que se va a enviar.
- `arduino.write(bytes(comando, 'utf-8'))`: Envía el comando como bytes a través de la conexión serial establecida.
- `time.sleep(0.05)`: Pausa la ejecución durante 0.05 segundos para asegurar que el comando se transmita correctamente.
- `ret`: Almacena la respuesta recibida desde el dispositivo externo.
- `arduino.readline()`: Lee una línea (respuesta) del dispositivo externo a través de la conexión serial.
- `print(ret)`: Muestra la respuesta recibida.

En esta sección del código, se ejecuta el envío de un comando a través de una conexión serial a un dispositivo externo, probablemente Arduino. El código envía el comando, espera un breve período y luego lee y muestra la respuesta recibida del dispositivo externo.

In [ ]:
comando = 'start'
arduino.write(bytes(comando,'utf-8'))
time.sleep(0.05)
ret = arduino.readline()
print(ret)


# Bucle de captura de video y detección


In [ ]:
# Iniciando bucle de captura de video
with mp_manos.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Limpiando el búfer serial
        arduino.flush()
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Volteando horizontalmente para visualización natural
        image = cv2.flip(image, 1)
        
        # Configurando la bandera de escritura
        image.flags.writeable = False
        
        # Detecciones de mano
        results = hands.process(image)
        
        # Configurando la bandera de escritura nuevamente
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Contando los dedos abiertos
        finger_count = 0
        gesture = ''  # Inicializando la variable de gesto
        
        # Renderizando los resultados
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                
                # Mediciones de distancias
                thumb_tip = hand_landmarks.landmark[mp_manos.HandLandmark.THUMB_TIP]
                thumb_ip = hand_landmarks.landmark[mp_manos.HandLandmark.THUMB_IP]
                pinky_mcp = hand_landmarks.landmark[mp_manos.HandLandmark.PINKY_MCP]
                
                thumb_tip_to_pinky_mcp = math.dist((thumb_tip.x, thumb_tip.y), (pinky_mcp.x, pinky_mcp.y))
                thumb_ip_to_pinky_mcp = math.dist((thumb_ip.x, thumb_ip.y), (pinky_mcp.x, pinky_mcp.y))
                
                # Pulgar
                if thumb_tip_to_pinky_mcp > thumb_ip_to_pinky_mcp:
                    finger_count += 1
                
                # Dedo índice
                if hand_landmarks.landmark[mp_manos.HandLandmark.INDEX_FINGER_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.INDEX_FINGER_PIP].y:
                    finger_count += 1
                
                # Dedo medio
                if hand_landmarks.landmark[mp_manos.HandLandmark.MIDDLE_FINGER_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.MIDDLE_FINGER_PIP].y:
                    finger_count += 1
                
                # Dedo anular
                if hand_landmarks.landmark[mp_manos.HandLandmark.RING_FINGER_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.RING_FINGER_PIP].y:
                    finger_count += 1
                
                # Dedo meñique
                if hand_landmarks.landmark[mp_manos.HandLandmark.PINKY_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.PINKY_PIP].y:
                    finger_count += 1
            
                mp_dibujo.draw_landmarks(image, hand_landmarks, mp_manos.HAND_CONNECTIONS, 
                                        mp_dibujo.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_dibujo.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
                
                # Predicción de gestos
                landmarks = np.array([[int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])] for lm in hand_landmarks.landmark])
                landmarks = np.expand_dims(landmarks, axis=0)
                prediction = model.predict(landmarks)
                classID = np.argmax(prediction)
                gesture = classNames[classID]
                
        if gesture == 'thumbs down':
            enviar_comando = False
            command = 'detener'
            arduino.write(b'detener')
            time.sleep(0.01)
            sent = False
            ret = arduino.readline()

            
        if gesture == 'thumbs up':
            enviar_comando = True
            command = 'start'
            arduino.write(command.encode())
            time.sleep(0.01)
            ret = arduino.readline()  
        sent = False
        
        if enviar_comando == True and finger_count == 10:
            # Enviar un comando para encender luces
            sent = True
            command = 'diez'
            arduino.write(b'diez')
            time.sleep(0.01)
            ret = arduino.readline()
            
        
        # Mostrar el conteo de dedos y la predicción de gesto
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, f'Conteo de Dedos: {finger_count}', (10, 30), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, f'Gesto: {gesture}', (10, 70), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, f'Enviar Habilitado: {enviar_comando} && Enviado {sent}', (10, 100), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

        
        # Mostrar el fotograma de salida final
        cv2.imshow('Seguimiento de Mano y Reconocimiento de Gesto', image)

        # Romper el bucle si se presiona la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Liberar la cámara web y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Close the serial connection when done
arduino.close()